# Modele generatywne; uczenie nienadzorowane

## Sieci kodujące-dekodujące

Jak wiemy z poprzednich zajęć sieci neuronowe pozwalają nam efektywnie zredukować wymiar wejściowego zbioru danych, zazwyczaj w celu odpowiedniej klasyfikacji i predykcji na zadanym zbiorze. Co jednak jeśli jesteśmy zainteresowani tylko redukcją wymiaru danych a nie wnioskowaniem na ich podstawie?

Klasyczna sieć kodująca-dekodująca (<b>autoencoder</b>) pozwala nam na zakodowanie danych (najczęściej obrazu) w formie wektora <b>zmiennych ukrytych</b> <i>(latent variables)</i> i następnie odtworzenie go w możliwie jak najbardziej bezstratnej formie:

[![](http://kvfrans.com/content/images/2016/08/autoenc.jpg)](http://kvfrans.com/variational-autoencoders-explained/)

Tego typu sieci posiadają kilka zastosowań:

- Są efektywnym narzędziem kompresji.
- Można je stosować jako sposób szyfrowania danych.
- Pozwalają na wykrycie nietrywialnych zależności między danymi, które nie są możliwe do uzyskania za pomocą klasycznych algorytmów uczenia nienadzorowanego.

Pokażmy prosty przykład:

In [1]:
using Flux, Flux.Data.MNIST
using Flux: mse, throttle, params, Statistics, @epochs
using Base.Iterators: partition
using Printf, BSON, LinearAlgebra

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv C:\Users\p\.julia\packages\CUDAdrv\mCr0O\src\CUDAdrv.jl:69


In [2]:
X = float.(hcat(vec.(MNIST.images())...))
N, M = size(X, 2), 100
data = [X[:,i] for i in Iterators.partition(1:N,M)];

In [3]:
encoder = Chain(Dense(28^2, 256, leakyrelu),
                Dense(256, 32, leakyrelu))
    
decoder = Chain(Dense(32, 256, leakyrelu), 
                Dense(256, 28^2, leakyrelu))

Chain(Dense(32, 256, leakyrelu), Dense(256, 784, leakyrelu))

In [12]:
model = Chain(encoder, decoder)
loss(x) = mse(model(x), x)
evalcb = throttle(() -> @show(loss(data[1])), 60)
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [11]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:10
    @info "Epoch $i"
    global best_acc, last_improvement
    Flux.train!(loss, params(model), zip(data), opt, cb = evalcb)
    ls = loss(data[1])
    if ls <= best_ls
        BSON.@save "enc_dec.bson" model epoch
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 2
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Beginning training loop...
└ @ Main In[11]:1


loss(data[1]) = 0.08695227443038495
loss(data[1]) = 0.05522200830117088
loss(data[1]) = 0.04271101870730842
loss(data[1]) = 0.03428420629914682
loss(data[1]) = 0.029653196647895563
loss(data[1]) = 0.026133213585833717
loss(data[1]) = 0.024004821294478503
loss(data[1]) = 0.0223705859977821
loss(data[1]) = 0.020996626711766702
loss(data[1]) = 0.01999302743946103
loss(data[1]) = 0.018960194715436956
loss(data[1]) = 0.018018079805186275
loss(data[1]) = 0.017086494928904718
loss(data[1]) = 0.017035316213796427
loss(data[1]) = 0.01656064963832795
loss(data[1]) = 0.01614566077469304
loss(data[1]) = 0.016111458475770273
loss(data[1]) = 0.015528620948371489
loss(data[1]) = 0.0151089517212893
loss(data[1]) = 0.01482919676125699
loss(data[1]) = 0.015002209191501832
loss(data[1]) = 0.01467366485890775
loss(data[1]) = 0.014209979337516498
loss(data[1]) = 0.014333477583144358
loss(data[1]) = 0.014066617111962214
loss(data[1]) = 0.013826348484915199
loss(data[1]) = 0.013686757888532573
loss(data[1]) 

┌ Warning:  -> We're calling this converged.
└ @ Main In[11]:15


In [43]:
using Images, ImageMagick

ArgumentError: ArgumentError: Package ImageMagick not found in current path:
- Run `import Pkg; Pkg.add("ImageMagick")` to install the ImageMagick package.


In [26]:
img(x::Vector) = Gray.(reshape(clamp.(x, 0, 1), 28, 28))

function sample()
  before = [MNIST.images()[i] for i in rand(1:length(MNIST.images()), 4)]
  after = img.(map(x -> model(float(vec(x))), before))
  vcat(hcat.(before, after)...)
end

sample (generic function with 1 method)

In [41]:
sample()

UndefVarError: UndefVarError: imshow not defined

Taki model pozwala na całkiem efektywną kompresję wejściowych danych. Jednak co należy zrobić gdy chcielibyśmy generować nowe obrazy na podstawie danych wejściowych a nie jedynie je odtwarzać?


##  Wariacyjne autoenkodery

[Variational Autoencoder](https://arxiv.org/pdf/1312.6114.pdf) jest klasą modeli, która na to pozwala. Intuicja  za nimi stojąca jest prosta, sieć dekodująca zamiast odtwarzać obraz w skali 1:1 dodaje do nich pewną losowość, dzięki której wyjściowy obraz jest nieznacznie zmieniony w stosunku do obrazu wejściowego.

Otrzymanie tej losowości jest możliwe dzięki wygenerowaniu przez sieć kodującą nie tyle wektora zmiennych ukrytych, co wektora średnich i odchyleń standardowych zmiennych ukrytych z których następnie losowane są zmienne ukryte wykorzystane przez sieć dekodującą:

[![](http://kvfrans.com/content/images/2016/08/vae.jpg)](http://kvfrans.com/variational-autoencoders-explained/)

Jak działa taka sieć?

Zgodnie z <b>hipotezą rozmaitości</b> (<i>Manifold hypothesis</i>) o kształcie  zbiorów danych o wysokim wymiarze decyduje tak naprawdę niewielka liczba zmiennych:

[![](https://www.researchgate.net/profile/Y_Bengio/publication/221700451/figure/fig1/AS:305523212734474@1449853818623/Example-of-a-simple-manifold-in-the-space-of-images-associated-with-a-rather-low-level.png)](https://www.researchgate.net/figure/Example-of-a-simple-manifold-in-the-space-of-images-associated-with-a-rather-low-level_fig1_221700451/)

Oznacza to, że do generowania dużyh zbiorów danych $x$ potrzebujemy tak naprawdę wiedzy jedynie o zmiennych ukrytych $z$, które je generują:

[![](https://i.stack.imgur.com/w0HP5.png)](https://stats.stackexchange.com/questions/271522/what-is-the-objective-of-a-variational-autoencoder-vae)

Tym co chcemy znaleźć jest rozkład prawdopodobieństwa $p(x,z)$, który wyjaśnia to w jaki sposób zmienne ukryte kształtują dane:

$$p(x,z) = p(x|z)p(z)$$

Mamy odwrotną informację:

$$p(z|x) = \frac{p(x|z)p(z)}{p(x)}$$

Aby dostać $p(x,z)$ musimy wyznaczyć $p(x) = \int {p(x|z)p(z)}dz$

Jak to zrobić? 
- bezpośrednio?
- generując dane z rozkladu i uśredniając wyniki?


Zamiast tego mozemy wyaproksymowac $p(z|x)$ za pomocą przyjętego rozkladu $Q(z|x; \theta)$, który należy do znanej nam rodziny rozkladów prawdopodobieństwa. Wtedy problem sprowadza się do policzenia::
$$E_{z \tilde{}  Q}[P(x | z)]$$ 


Jak jednak $Q$ ma sie do $p$?


Mozemy to zmierzyc za pomoca Dywergencji Kullbacka-Leiblera:

$$d_{KL}(Q(z|x)||p(z  |  x)) = E_{Q}[\log Q(z|x)] - E_{Q}[\log p(x,z)] + \log p(x)$$

Za zadanie mamy znaleźć takie $Q$, które zminimalizuje dywergencje 


Aby to zrobić zdefiniujmy <b>Evidence Lower BOund</b> (ELBO):

$$ELBO(\theta) = E_{Q}[\log p(x,z)] -  E_{Q}[\log Q(z|x)]$$

Zauważmy, że:

$$\log p(x) = ELBO(\theta) + d_{KL}(Q(z|x)||p(z  |  x))$$

Na mocy nierówności Jensena Dywergencja Kullbacka-Leiblera jest zawsze nieujemna, co oznacza, że problem minimalizacji dywergencji sprowadza się do problemu maksymalizacji ELBO.

Przejdźmy do implementacji modelu:

In [44]:
X = float.(hcat(vec.(MNIST.images())...)) .> 0.5
N, M = size(X, 2), 100
data = [X[:,i] for i in Iterators.partition(1:N,M)];

Zdefiniujmy funkcje straty:

In [45]:
# Extend distributions slightly to have a numerically stable logpdf for `p` close to 1 or 0.
using Distributions
import Distributions: logpdf
logpdf(b::Bernoulli, y::Bool) = y * log(b.p + eps(Float32)) + (1f0 - y) * log(1 - b.p + eps(Float32))

logpdf (generic function with 64 methods)

In [46]:
# KL-divergence between approximation posterior and N(0, 1) prior
kl_q_p(μ, logσ) = 0.5f0 * sum(exp.(2f0 .* logσ) + μ.^2 .- 1f0 .+ logσ.^2)

kl_q_p (generic function with 1 method)

In [47]:
# logp(x|z) - conditional probability of data given latents.
logp_x_z(x, z) = sum(logpdf.(Bernoulli.(decoder(z)), x))

logp_x_z (generic function with 1 method)

In [48]:
function ELBO(X)
    μ̂, logσ̂ = μ(encoder(X)), logσ(encoder(X))
    logp_x_z(X, z.(μ̂, logσ̂)) - kl_q_p(μ̂, logσ̂) * 1 // M
end

ELBO (generic function with 1 method)

In [49]:
loss(X) = -ELBO(X) 

loss (generic function with 1 method)

I model:

In [50]:
Dz, Dh = 2, 512 #wymiary

(2, 512)

In [51]:
encoder  = Chain(Dense(28^2, 512, relu))

Chain(Dense(784, 512, relu))

In [52]:
μ = Dense(Dh, Dz)
logσ = Dense(Dh, Dz)

Dense(512, 2)

In [53]:
z(μ, logσ) = μ + exp(logσ) * randn(Float32)

z (generic function with 1 method)

In [54]:
decoder = Chain(Dense(Dz, Dh, relu), 
                Dense(Dh, 28^2, σ))

Chain(Dense(2, 512, relu), Dense(512, 784, σ))

In [57]:
evalcb = throttle(() -> @show(loss(X[:, rand(1:N, M)])), 60)
opt = ADAM()
ps = params(encoder, μ, logσ, decoder);

## @info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:10
    @info "Epoch $epoch"
    global best_acc, last_improvement
    Flux.train!(loss, ps, zip(data), opt, cb=evalcb)
    ls = loss(data[1])
    if ls <= best_ls
        BSON.@save "vae.bson" encoder μ logσ decoder
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 2
        @warn(" -> We're calling this converged.")
        break
    end
end

Do budowania modeli generatywnych możemy wykorzystać też sieci konwolucyjne:

In [ ]:
Dz, Dh = 6, 128 #wymiary

In [ ]:
encoder =  Chain(x -> reshape(x, (28,28,1,100)),
        Conv((3,3), 1=>32, pad = (1,1),relu),
        Conv((3,3), 32=>32, pad = (1,1), relu),
        MaxPool((2, 2)),
        Dropout(0.25),
        x -> reshape(x, :, size(x, 4)),
        Dense(6272, Dh))

μ = Dense(Dh, Dz) 

logσ = Dense(Dh, Dz)

z(μ, logσ) = μ + exp.(logσ) * randn(Float32)

decoder = Chain(Dense(Dz, Dh, relu),
        Dense(Dh,6272, relu),
        x -> reshape(x,(14,14,32,100)),
        Dropout(0.25),
        x -> repeat(x, inner = [2,2,1,1]),
        ConvTranspose((3,3), 32=>32, pad = (1,1),relu),
        ConvTranspose((3,3), 32=>1, pad = (1,1), σ),
        x -> reshape(x, :, size(x, 4))
       )

In [ ]:
evalcb = throttle(() -> @show(loss(X[:, rand(1:N, M)])), 300)
opt = ADAM()
ps = params(encoder, μ, logσ, decoder);

In [ ]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:10
    @info "Epoch $epoch"
    global best_acc, last_improvement
    Flux.train!(loss, ps, zip(data), opt, cb=evalcb)
    ls = loss(data[1])
    if ls <= best_ls
        BSON.@save "vaeCNN_MNIST.bson" encoder μ logσ decoder
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 2
        @warn(" -> We're calling this converged.")
        break
    end
end

In [ ]:
BSON.@load "vaeCNN_MNIST.bson" encoder μ logσ decoder 

In [ ]:
img(x::Vector) = Gray.(reshape(clamp.(x, 0, 1), 28, 28))

function sample()
    batch = data[rand(1:length(data))]
    latents = encoder(batch)
    Z = z(μ(latents), logσ(latents))
    decoded = decoder(Z)
    idx = rand(1:size(batch,2),4)
    before = [img(batch[:,i]) for i in idx]
    after = [img(decoded[:,i]) for i in idx]
    vcat(hcat.(before, after)...)
end

In [ ]:
sample()